In [1]:
import os
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# torch imports
import torch
import torch.nn as nn

## CARLA dataset

In [2]:
from config import GlobalConfig
from data import CARLA_Data

root_dir = '/home/surya/Downloads/transfuser-2022/data/'
config = GlobalConfig(root_dir=root_dir, setting='all')
train_set = CARLA_Data(root=config.train_data, config=config)
val_set = CARLA_Data(root=config.val_data, config=config)
print(f"There are {len(train_set)} samples in training set")

100%|██████████| 11/11 [00:01<00:00,  7.78it/s]
Loading 16088 lidars from 11 folders
100%|██████████| 8/8 [00:00<00:00,  9.15it/s]
Loading 11543 lidars from 8 folders
There are 16088 samples in training set


Create pytorch style dataloaders

In [3]:
from torch.utils.data import DataLoader

g_cuda = torch.Generator(device='cpu')
g_cuda.manual_seed(torch.initial_seed())

# We need to seed the workers individually otherwise random processes 
# in the dataloader return the same values across workers!
def seed_worker(worker_id):
    # Torch initial seed is properly set across the different workers,
    # we need to pass it to numpy and random.
    worker_seed = (torch.initial_seed()) % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


dataloader_train = DataLoader(train_set, shuffle=True, batch_size=2, worker_init_fn=seed_worker, generator=g_cuda, num_workers=4)
dataloader_val   = DataLoader(val_set, shuffle=False, batch_size=2, worker_init_fn=seed_worker, generator=g_cuda, num_workers=4)

In [4]:
sample_data = next(iter(dataloader_train))
print(f"sample data is of type {type(sample_data)} and has following keys")

for k,v in sample_data.items():
    print(k, list(v.shape))

sample data is of type <class 'dict'> and has following keys
rgb [2, 3, 160, 704]
bev [2, 160, 160]
depth [2, 160, 704]
semantic [2, 160, 704]
steer [2]
throttle [2]
brake [2]
speed [2]
theta [2]
x_command [2]
y_command [2]
light [2]
target_point [2, 2]
target_point_image [2, 1, 256, 256]
lidar [2, 2, 256, 256]
label [2, 20, 7]
ego_waypoint [2, 4, 2]


## Model

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

In [7]:
from model import LidarCenterNet
model = LidarCenterNet(config, device, config.backbone, image_architecture='regnety_032', 
                           lidar_architecture='regnety_032', use_velocity=False)
model.to(device);

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print ('Total trainable parameters: ', params)

Total trainable parameters:  168018327


## Load pretrained model

In [10]:
# from collections import OrderedDict
# new_state_dict = OrderedDict()

# for k,v in checkpt.items():
#     new_key = k.replace("module.", "")
#     if new_key != '_model.lidar_encoder._model.stem.conv.weight':
#         new_state_dict[new_key] = v
# torch.save(new_state_dict, 'transfuser_regnet032_seed1_39.pth')

In [11]:
model.eval();
checkpt = torch.load('/home/surya/Downloads/transfuser-2022/model_ckpt/transfuser/transfuser_regnet032_seed1_39.pth', map_location=device)
model.load_state_dict(checkpt)

<All keys matched successfully>

In [14]:
model.config.debug = True
test_data = next(iter(dataloader_val))

In [15]:
# load data to gpu, according to type
for k in ['rgb', 'depth', 'lidar', 'label', 'ego_waypoint', \
          'target_point', 'target_point_image', 'speed']:
    test_data[k] = test_data[k].to(device, torch.float32)
for k in ['semantic', 'bev']:
    test_data[k] = test_data[k].to(device, torch.long)

In [16]:
# losses = model(test_data, save_path='.')

/home/surya/miniconda3/envs/tfuse/lib/python3.7/site-packages/mmdet/models/utils/gaussian_target.py:227: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  topk_clses = topk_inds // (height * width)
/home/surya/miniconda3/envs/tfuse/lib/python3.7/site-packages/mmdet/models/utils/gaussian_target.py:229: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='fl